In [ ]:
import numpy as np
import graphlab as gl
from graphlab.toolkits.feature_engineering import *
encoder = WordCounter(delimiters=["\r", "\v", "\n", "\f", "\t", " ",'"',"'",',','.','!',';',':','$','%','-','_','/','\\'
                                  ,'(',')','?','[',']','{','}'
                                 ,'0','1','2','3','4','5','6','7','8','9'])
import gensim
w2v = gensim.models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)
def getvec(y):
    try: 
        return w2v[y]
    except:
        return []
def getvec2(x):
    try:
        return np.mean([z for z in [getvec(y) for y in x.keys()] if z != []], axis=0)
    except: 
        return None


def producebow(sfx,tstr):
    transformed_sf = encoder.fit_transform(sfx[[tstr]])
    transformed_sf[tstr] = transformed_sf[tstr].dict_trim_by_keys(gl.text_analytics.stopwords(), True)
    return transformed_sf[tstr]
def producevec(sfx,tstr):
    transformed_df = sfx.copy()
    transformed_sf[tstr] = [getvec2(x) for x in transformed_df[tstr]]
    return transformed_sf[tstr]

In [2]:
def producebow(sfx,tstr):
    transformed_sf = encoder.fit_transform(sfx[[tstr]])
    transformed_sf[tstr] = transformed_sf[tstr].dict_trim_by_keys(gl.text_analytics.stopwords(), True)
    return transformed_sf[tstr]
def producevec(sfx,tstr):
    return [getvec2(x) for x in sfx[tstr]]

In [3]:
inp = gl.SFrame()
inp['text']=[ open('moi.txt', 'r').read().decode('ascii','ignore')]

In [4]:
inp['text'][0]

'Enabling mobility in neglected areas of the world\nMobility Outreach International (formerly Prosthetics Outreach Foundation)enables mobility in neglected areas of the world and has served over 21,000 adults and children immobilized by limb loss and deformity. Mobility Outreach Internationals programs deploy expertise in prosthetics, orthotics and clubfoot treatment to offer advocacy, training, rehabilitation services and occasional orthopedic surgery.'

In [5]:
inp['bow']=producebow(inp,'text')
inp['vector'] = producevec(inp,'bow')

In [6]:
data = gl.SFrame('mystory')

In [7]:
data['Story_Text'] = data['Story_Text'].apply(lambda x: x.decode('ascii','ignore'))
data['1gram']=producebow(data[['Story_Text']],'Story_Text')
data['vector'] = producevec(data[['1gram']],'1gram')

In [8]:
data['perc'] = data['Total Expenses']/data['Total Revenue']

In [9]:
tokenizer = Tokenizer()

In [10]:
tokenized_sf = tokenizer.fit_transform(data[['Story_Text']])

In [11]:
tokenized_sf['len'] = tokenized_sf['Story_Text'].apply(lambda x: len(x))

In [12]:
data['words'] = tokenized_sf['len']

In [13]:
data['target'] = data['votes_total']>40

In [14]:
sf_train, sf_test = data.random_split(.9, seed=5)

In [16]:
data.column_names()

['NonProfitOrg_Guidestar_Ver',
 'Inclusion',
 'votes_total',
 'votes_gallery_(Calc)',
 'votes_local_network',
 'createdate',
 'ipaddress',
 'EIN',
 'facebook_name',
 'gender',
 'location',
 'language',
 'position',
 'affiliation',
 'email_Submmitted',
 'Photo',
 'URL_in_Story_(Calc)',
 'matched_zip',
 'matched_city',
 'matched_state',
 'Total Revenue',
 'Total Expenses',
 'NTEE_Code',
 'Story_Text',
 'give_reason',
 'give_method',
 'X27',
 '.1',
 '.2',
 '.3',
 '.4',
 '.5',
 '.6',
 'text length',
 'remaining',
 '1gram',
 'vector',
 'perc',
 'words',
 'target']

In [20]:
 model = gl.boosted_trees_classifier.create(sf_train, target='target',
                                 features=
                                     ['Photo'
                                      ,'text length'
                                      ,'NTEE_Code'
                                     ,'perc'
                                    ,'remaining'
                                    ,'Total Revenue'
                                      ,'Total Expenses'
                                      ,'Inclusion'
                                      ,'words'
                                      ,'1gram'
                                      ,'give_reason'
                                      ,'give_method'
                                      ,'vector'
                                      #,'matched_state'
                                      #,'SentimentScore',
                                      #,'NumWords',
                                      
 #'NumPunctuation',
 #'NumURLs',
 #'NumHashtags',
 #'NumAtMentions',
 #'NumDollarsigns',
 #'NumExclamationpoints',
 #'NumDigits',
 #'NumUppercaseletters',
 #'EmailDomain',
 #'EmailTLD',
 #'ScrabbleScore',
 #'InferredGender'
                                    ]
                                    , validation_set=sf_test
                                    ,max_iterations=10
                                    ,max_depth=6)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 736

Number of classes           : 2

...

| 3         | 0.119998     | 0.951087          | 0.689655            | 0.351349          | 0.568810            |

| 4         | 0.156998     | 0.959239          | 0.712644            | 0.300050          | 0.561278            |

| 5         | 0.195998     | 0.970109          | 0.712644            | 0.258956          | 0.550639            |

| 6         | 0.232999     | 0.978261          | 0.724138            | 0.228286          | 0.549185            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [21]:
model.get_feature_importance()

name,index,count
text length,None,17
Photo,TRUE,10
1gram,family,4
vector,164,4
words,None,4
1gram,calls,3
vector,235,3
1gram,america,3
1gram,programs,3
1gram,close,3


In [22]:
model.evaluate(sf_test)

{'accuracy': 0.735632183908046,
 'auc': 0.6634328358208952,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        |   5   |
 |      0       |        1        |   8   |
 |      1       |        0        |   15  |
 |      0       |        0        |   59  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.30303030303030304,
 'log_loss': 0.5578075314505592,
 'precision': 0.38461538461538464,
 'recall': 0.25,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+----+----+
 | threshold | fpr | tpr | p  | n  |
 +-----------+-----+-----+----+----+
 |    0.0    | 1.0 | 1.0 | 20 | 67 |
 |   1e-05   | 1.0 | 1.0 | 20 | 67 |
 |   2e-05   | 1.0 | 1.0 | 20 | 67 |
 |   3e-05   | 

In [51]:
sk = gl.Sketch(tokenized_sf['len'])
[sk.min()
 ,sk.quantile(0.25)
 ,sk.quantile(0.5)
 ,sk.quantile(0.75)
,sk.max()]

[15.0, 195.0, 278.0, 446.0, 4911.0]

In [78]:
sk = gl.Sketch(data['votes_total'])
[sk.min()
 ,sk.quantile(0.25)
 ,sk.quantile(0.5)
 ,sk.quantile(0.75)
,sk.max()]

[0.0, 2.0, 7.0, 41.0, 14812.0]

In [174]:
data['give_reason']=data['give_reason'].apply(lambda x: str(x)) 
data['give_method']=data['give_method'].apply(lambda x: str(x)) 

In [88]:
data['target'] = data['votes_total']>40

In [178]:
h = len(data[data['target']==1])
d = len(data[data['target']==0])

In [180]:
OR = data[['target','1gram']].stack('1gram')

In [181]:
OR['X2'] = OR['X2'].apply(lambda x: x.strip())

In [182]:
OR = OR.groupby('X2',{'he':gl.aggregate.SUM('target'),'total':gl.aggregate.COUNT})

In [183]:
OR['de'] = OR['total']-OR['he']

In [184]:
OR['hn'] = h-OR['he']

In [185]:
OR['dn'] = d-OR['de']

In [186]:
import scipy as sc
OR['odds'] = OR.apply(lambda x: sc.stats.fisher_exact([[x['de'],x['dn']],[x['he'],x['hn']]]))
OR = OR.unpack('odds')


In [187]:
OR[OR['odds.1']<=0.01].sort('odds.0').save('wordsodds.csv')

In [138]:
OR[OR['odds.1']<0.05]['X2'].unique()

dtype: str
Rows: 8
[' G96', '', 'H12', 'D34', ' E01', 'G84', ' H12', 'E99']

In [140]:
' f'.strip()

'f'

In [3]:
import gensim
w2v = gensim.models.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
w2v.most_similar('bird')

[(u'birds', 0.8141970634460449),
 (u'raptor', 0.6830928325653076),
 (u'owl', 0.6825829148292542),
 (u'squirrel', 0.6653631329536438),
 (u'falcon', 0.6649248600006104),
 (u'raptors', 0.6613034605979919),
 (u'bald_eagle', 0.6493616104125977),
 (u'robin', 0.6482654809951782),
 (u'pelican', 0.6474649310112),
 (u'avian', 0.6436793208122253)]

In [8]:
import numpy as np

In [104]:
kmodel = gl.nearest_neighbors.create(data, features=['vector'],distance='cosine')

Starting brute force nearest neighbors model training.

In [148]:
data = data.add_row_number()

In [150]:
data[['id','target']].save('storyvert.csv')

In [147]:
kmodel.similarity_graph().edges.save('storyedg.csv')


Starting blockwise similarity graph construction.

max rows per data block: 7893

number of reference data blocks: 4

+---------+-------------+--------------+

| # Pairs | % Complete. | Elapsed Time |

+---------+-------------+--------------+

| 126691  | 29.8546     | 9.999ms      |

| 424360  | 100         | 39ms         |

+---------+-------------+--------------+

In [137]:
kmodel.query(inp, k=2).tail()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.121507    | 1ms          |

| Done         |         | 100         | 7.999ms      |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,686,0.271525086782,1
0,599,0.27159025233,2


In [139]:
interests = {'health':1
             ,'women':1
             ,'education':1
            }
sft = gl.SFrame()
sft['vector'] = [getvec2(interests)]
kmodel.query(sft, k=2).tail()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.121507    | 1.001ms      |

| Done         |         | 100         | 8ms          |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,520,0.35133983728,1
0,30,0.399923659875,2


In [132]:
which = 520
kmodel.query(data[which:which+1], k=2).tail()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.121507    | 1ms          |

| Done         |         | 100         | 7.001ms      |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,520,0.0,1
0,646,0.12341007222,2


In [141]:
which = 646
data['Story_Text'][which], data['votes_total'][which]

("Time, Talent, Treasure\n\nAbout 35 years ago I found an organization that I feel is doing important work in our community.  At the time, women who were pregnant out of marriage were given few choices and often had to 'go live with an aunt' to protect the family's reputation in their community.  Fortunately this stigma has changed in our society, but the problems a woman faces in a crisis pregnancy have not.\n\nWhen Crisis Pregnancy Outreach (www.cpotulsa.org) began in the early 1980s, many of the women who came to us were teenagers who did not want to have an abortion.  Today, CPO works in coordination with local hospitals and operates a 24 hour hotline.  The majority of woman we now serve are homeless; addicted to drugs/alcohol, gambling, or relationships; physically abused; and/or are totally alone.  \n\nWhat I love about CPO is that we make a lifetime commitment to these women.  It is not just about the 9 months of pregnancy. This crisis brings women to CPO, but this is only the p

In [138]:
which =686
data['Story_Text'][which], data['votes_total'][which]

('My name is Jenny Oveson, Im 28 years old, and I was born with Spina Bifida.  I wanted to write you to tell you about an organization that has grown to mean so much to me.  As a young child, I watched my father make it his mission in life to better the lives of people with disabilities and our families.  He is achieving this as Director of Family Support.  Family Support, the home of the Individual & Family Support program in south Alabama provides financial assistance for medical, dental, prescriptions, supplies, utilities, food, clothing, housing, holiday gifts, ramps, home modifications/repairs, transportation costs including auto repairs, conference expenses, summer camps, education, respite, recreation, etc.  Also information, moral support and referrals may be provided to help families access other programs of assistance.  Family Support works to enhance the lives of Alabamians with disabilities and their families.  Their success is measured in the number of people they can help